In [2]:
import os
os.getcwd()

'/home/aniemcz/gooseReno'

# Run RENO, pretrained on Goose, on goose dataset

In [3]:
reno_path = "./RENO/"
ckpt_path = os.path.join(reno_path, "model", "Goose", "ckpt.pt")
input_glob = "RENO/data/goose_examples/*.ply"
output_folder = "RENO/data/goose_compressed/"
decomp_folder = "RENO/data/goose_decompressed/"
posQ = 128

In [4]:
!pixi run python RENO/compress.py \
    --input_glob='{input_glob}' \
    --output_folder='{output_folder}' \
    --ckpt='{ckpt_path}' \
    --posQ={posQ}

⠁ activating environment                                                        Using /home/aniemcz/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Emitting ninja build file /home/aniemcz/.cache/torch_extensions/py310_cu118/torchac_backend/build.ninja...
Building extension module torchac_backend...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module torchac_backend...
Loading point clouds...
100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.64it/s]
Traceback (most recent call last):
  File "/home/aniemcz/gooseReno/RENO/compress.py", line 68, in <module>
    net(SparseTensor(coords=torch.cat((random_coords[:, 0:1]*0, random_coords), dim=-1),
  File "/home/aniemcz/gooseReno/.pixi/envs/default/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/a

In [4]:
!pixi run python RENO/decompress.py \
    --input_glob='{output_folder}*.bin' \
    --output_folder='{decomp_folder}' \
    --ckpt='{ckpt_path}'

⠁ activating environment                                                        Using /home/aniemcz/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Emitting ninja build file /home/aniemcz/.cache/torch_extensions/py310_cu118/torchac_backend/build.ninja...
Building extension module torchac_backend...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module torchac_backend...
100%|█████████████████████████████████████████████| 3/3 [00:03<00:00,  1.04s/it]
Total: 3 | Decode Time:0.289 | Max GPU Memory:487.89MB


In [43]:
import os
import glob

# Set your glob pattern here
og_file_paths = glob.glob(input_glob, recursive=True)
compressed_file_paths = glob.glob(os.path.join(output_folder, "*.bin"), recursive=True)

original_total_size_bytes = sum(os.path.getsize(f) for f in og_file_paths if os.path.isfile(f))
compressed_total_size_bytes = sum(os.path.getsize(f) for f in compressed_file_paths if os.path.isfile(f))

print(f"Total size: original {original_total_size_bytes} bytes and \
      compressed {compressed_total_size_bytes} bytes for posQ = {posQ}")

print(f"Compress Ratio: { original_total_size_bytes / compressed_total_size_bytes }")

Total size: original 14375302 bytes and       compressed 742121 bytes for posQ = 8
Compress Ratio: 19.370563560389748


In [44]:
#14375302

In [ ]:
'''
Goose on 3 files:
Original Total Size of the 3 files: 14375302
QP | Total: Files | Avg. Bpp: Num   | Encode time: Sec  | Max GPU Memory: MB        | Compression Ratio | Comp Size
8  | Total: 3     | Avg. Bpp:10.434 | Encode time:0.208 | Max GPU Memory:559.12MB   | 19.37             | 742121
16 | Total: 3     | Avg. Bpp:7.780  | Encode time:0.160 | Max GPU Memory:525.03MB   | 25.77             | 557836
32 | Total: 3     | Avg. Bpp:5.425  | Encode time:0.118 | Max GPU Memory:436.84MB   | 36.52             | 393672
64 | Total: 3     | Avg. Bpp:3.452  | Encode time:0.087 | Max GPU Memory:304.72MB   | 57.00             | 252237
128| Total: 3     | Avg. Bpp:1.972  | Encode time:0.060 | Max GPU Memory:170.15MB   | 100.46            | 143096
256| Total: 3     | Avg. Bpp:0.985  | Encode time:0.039 | Max GPU Memory:80.76MB    | 203.97            | 70478
512| Total: 3     | Avg. Bpp:0.422  | Encode time:0.031 | Max GPU Memory:47.31MB    | 489.69            | 29356
'''

### Print out compression Ratio of RENO

In [48]:
#og_ply_path = "./data/og/exaalt-copper-ds1-fr0.ply"
#cmp_ply_path = "./data/compressed/reno/exaalt-copper-ds1-fr0.ply.bin"

#os.path.getsize returns size in bytes
#og_ply_size = os.path.getsize(og_ply_path)
#decmp_ply_size = os.path.getsize(decmp_ply_path)

#print(f"Compression ratio is {og_ply_size / decmp_ply_size}")

Compression ratio is 1.9963850418895484


# Run our custom PSNR script on RENO

In [1]:
og_ply_path = 1

In [49]:
!source activate lidarDataVisualEnv && \
python3 psnr_ply.py {og_ply_path} ./data/decompressed/reno/exaalt-copper-ds1-fr0.ply.bin.ply

MSE: 323.706607
PSNR: 8.67 dB


In [ ]:
import nump as np
